## Conectarse a MySQL mediante Python

In [1]:
import mysql.connector
from mysql.connector import Error

Lo que primero se necesita es un diccionario para la configuracion de la conexion.

In [2]:
config = {
    'user': 'root',          # Usuario de MySQL
    'password': 'mascotas12@',  # Contraseña de MySQL
    'host': '127.0.0.1',     # Host (localhost)
    'database': 'mascotas',  # Nombre de la base de datos
    'raise_on_warnings': True
}

Ahora ejecutaremos el script para insertar datos

In [4]:
try:
    # Conectar a la base de datos
    conexion = mysql.connector.connect(**config)

    if conexion.is_connected():
        print("Conexión exitosa a la base de datos")

        # Crear un cursor para ejecutar consultas
        cursor = conexion.cursor()

        # Crear una tabla si no existe
        crear_tabla = """
        CREATE TABLE IF NOT EXISTS codigo_postal (
            cod_postal INT PRIMARY KEY,
            cantidad_casos INT DEFAULT 0,
            latitud FLOAT,
            longitud FLOAT
        );

        CREATE TABLE IF NOT EXISTS codigo_postal_adyacente (
            cod_postal_origen INT,
            cod_postal_adyacente INT,
            PRIMARY KEY (cod_postal_origen, cod_postal_adyacente),
            FOREIGN KEY (cod_postal_origen) REFERENCES codigo_postal(cod_postal),
            FOREIGN KEY (cod_postal_adyacente) REFERENCES codigo_postal(cod_postal)
        );

        CREATE TABLE IF NOT EXISTS mascota (
            id_mascota SERIAL PRIMARY KEY,
            ciudad VARCHAR(50),
            edad INT,
            sexo VARCHAR(10)  -- Campo crítico para consultas como "machos"
        );

        CREATE TABLE IF NOT EXISTS test (
            id_test SERIAL PRIMARY KEY,
            cod_postal INT REFERENCES codigo_postal(cod_postal),
            fecha DATE NOT NULL,
            enfermedad VARCHAR(50) NOT NULL,
            id_mascota INT REFERENCES mascota(id_mascota),
            resultado BOOLEAN NOT NULL  -- TRUE = positivo, FALSE = negativo
        );
        """
        cursor.execute(crear_tabla)

        # Insertar datos en la tabla
        insertar_datos = """
        INSERT INTO mascotas (nombre, edad, especie)
        VALUES (%s, %s, %s)
        """
        datos = [
            ("Firulais", 3, "Perro"),
            ("Michi", 2, "Gato"),
            ("Piolín", 1, "Pájaro")
        ]

        cursor.executemany(insertar_datos, datos)
        conexion.commit()  # Confirmar la transacción
        print(f"{cursor.rowcount} registros insertados")

except Error as e:
    print(f"Error al conectarse a MySQL: {e}")

finally:
    # Cerrar la conexión
    if conexion.is_connected():
        cursor.close()
        conexion.close()
        print("Conexión cerrada")

Conexión exitosa a la base de datos
Error al conectarse a MySQL: Failed getting warnings; 2014 (HY000): Commands out of sync; you can't run this command now
